In [1]:
# importing libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score

In [2]:
# reading train and test datasets
train_data = pd.read_csv("/kaggle/input/spaceship-titanic/train.csv", index_col='PassengerId')
test_data = pd.read_csv("/kaggle/input/spaceship-titanic/test.csv", index_col='PassengerId')

train_data.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
PassengerId,,,,,,,,,,,,,
0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [3]:
test_data.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
PassengerId,,,,,,,,,,,,
0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez


In [4]:
display(train_data.info())
display(train_data.nunique())

<class 'pandas.core.frame.DataFrame'>
Index: 8693 entries, 0001_01 to 9280_02
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    8492 non-null   object 
 1   CryoSleep     8476 non-null   object 
 2   Cabin         8494 non-null   object 
 3   Destination   8511 non-null   object 
 4   Age           8514 non-null   float64
 5   VIP           8490 non-null   object 
 6   RoomService   8512 non-null   float64
 7   FoodCourt     8510 non-null   float64
 8   ShoppingMall  8485 non-null   float64
 9   Spa           8510 non-null   float64
 10  VRDeck        8505 non-null   float64
 11  Name          8493 non-null   object 
 12  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(6)
memory usage: 891.4+ KB


None

HomePlanet         3
CryoSleep          2
Cabin           6560
Destination        3
Age               80
VIP                2
RoomService     1273
FoodCourt       1507
ShoppingMall    1115
Spa             1327
VRDeck          1306
Name            8473
Transported        2
dtype: int64

In [5]:
# select predictors
cols_to_use = ['HomePlanet','CryoSleep', 'Cabin', 'Destination', 'Age', 'VIP', 'RoomService','FoodCourt',
'ShoppingMall', 'Spa', 'VRDeck']
X = train_data[cols_to_use]
X_test = test_data[cols_to_use]

# select target
y = train_data.Transported

# splitting training data into training and validation subsets
X_train, X_valid, y_train, y_valid = train_test_split(X,y,train_size=0.8)

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6954 entries, 1148_07 to 5756_07
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    6793 non-null   object 
 1   CryoSleep     6785 non-null   object 
 2   Cabin         6799 non-null   object 
 3   Destination   6808 non-null   object 
 4   Age           6808 non-null   float64
 5   VIP           6787 non-null   object 
 6   RoomService   6807 non-null   float64
 7   FoodCourt     6811 non-null   float64
 8   ShoppingMall  6793 non-null   float64
 9   Spa           6809 non-null   float64
 10  VRDeck        6814 non-null   float64
dtypes: float64(6), object(5)
memory usage: 651.9+ KB


In [7]:
#extracting numeric columns for further encoding and imputing missing values
num_cols_train = [col for col in X_train if X_train[col].dtype in ['float64']]
num_cols_valid = [col for col in X_valid if X_valid[col].dtype in ['float64']]
num_cols_test = [col for col in X_test if X_test[col].dtype in ['float64']]

num_cols_train = X_train[num_cols_test]
num_cols_valid = X_valid[num_cols_valid]
num_cols_test = X_test[num_cols_test]

num_c_train, num_i_train = num_cols_train.columns, num_cols_train.index
num_c_valid, num_i_valid = num_cols_valid.columns, num_cols_valid.index
num_c_test, num_i_test = num_cols_test.columns, num_cols_test.index

num_imputer = SimpleImputer(strategy='median')
num_cols_train = pd.DataFrame(num_imputer.fit_transform(num_cols_train,y_train))
num_cols_valid= pd.DataFrame(num_imputer.transform(num_cols_valid))
num_cols_test = pd.DataFrame(num_imputer.transform(num_cols_test))

num_cols_train.columns, num_cols_train.index = num_c_train, num_i_train
num_cols_valid.columns, num_cols_valid.index = num_c_valid, num_i_valid
num_cols_test.columns, num_cols_test.index = num_c_test, num_i_test

#imputing missing values
imputer = SimpleImputer(strategy='most_frequent')

train_cols=X_train.columns

train_index = X_train.index
valid_cols=X_valid.columns
valid_index=X_valid.index
test_cols=X_test.columns
test_index=X_test.index

X_train=pd.DataFrame(imputer.fit_transform(X_train, y_train))

X_valid=pd.DataFrame(imputer.transform(X_valid))

X_test=pd.DataFrame(imputer.transform(X_test))

X_train.columns = train_cols
X_train.index = train_index
X_valid.columns = valid_cols
X_valid.index = valid_index
X_test.columns = test_cols
X_test.index = test_index

#inserting numeric columns back
X_train[num_c_train] = num_cols_train
X_valid[num_c_valid] = num_cols_valid
X_test[num_c_test] = num_cols_test

In [8]:
#splitting Cabin column
train_cab_expanded = X_train['Cabin'].str.split('/',expand=True)
X_train[['Cab_1', 'Cab_2', 'Cab_3']] = train_cab_expanded

valid_cab_expanded = X_valid['Cabin'].str.split('/',expand=True)
X_valid[['Cab_1', 'Cab_2', 'Cab_3']] = valid_cab_expanded

test_cab_exp = X_test['Cabin'].str.split('/',expand=True)
X_test[['Cab_1', 'Cab_2', 'Cab_3']] = test_cab_exp

X_train['Cab_2']=pd.to_numeric(X_train['Cab_2'], errors='coerce')
X_valid['Cab_2']=pd.to_numeric(X_valid['Cab_2'], errors='coerce')
X_test['Cab_2']=pd.to_numeric(X_test['Cab_2'], errors='coerce')

X_train=X_train.drop('Cabin', axis=1)
X_valid=X_valid.drop('Cabin', axis=1)
X_test=X_test.drop('Cabin', axis=1)

In [9]:
X_train.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Cab_1,Cab_2,Cab_3
PassengerId,,,,,,,,,,,,,
1148_07,Europa,False,55 Cancri e,34.0,False,2386.0,841.0,76.0,26.0,1693.0,C,42,P
7255_01,Earth,False,TRAPPIST-1e,10.0,False,0.0,0.0,0.0,0.0,0.0,G,1166,P
1736_01,Earth,True,PSO J318.5-22,22.0,False,0.0,0.0,0.0,0.0,0.0,G,284,P
4239_01,Earth,False,TRAPPIST-1e,68.0,False,0.0,1.0,223.0,580.0,0.0,F,875,P
1928_01,Earth,False,PSO J318.5-22,21.0,False,0.0,0.0,107.0,166.0,388.0,E,144,S


In [10]:
#num_cols = [col for col in X if X[col].dtype in ['int64', 'float64']]

low_card_cols_train = [col for col in X_train if X_train[col].nunique() < 10 and
X_train[col].dtype == 'object']
low_card_cols_valid = [col for col in X_valid if X_valid[col].nunique() < 10 and
X_valid[col].dtype == 'object']
low_card_cols_test = [col for col in X_test if X_test[col].nunique() < 10 and
X_test[col].dtype == 'object']

#applying one hot encoding to low cardinality categorical columns
X_train = pd.get_dummies(X_train,columns=low_card_cols_train)
X_valid = pd.get_dummies(X_valid, columns=low_card_cols_valid)
X_test = pd.get_dummies(X_test,columns=low_card_cols_test)

In [11]:
X_valid.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1739 entries, 4010_01 to 1071_01
Data columns (total 27 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Age                        1739 non-null   float64
 1   RoomService                1739 non-null   float64
 2   FoodCourt                  1739 non-null   float64
 3   ShoppingMall               1739 non-null   float64
 4   Spa                        1739 non-null   float64
 5   VRDeck                     1739 non-null   float64
 6   Cab_2                      1739 non-null   int64  
 7   HomePlanet_Earth           1739 non-null   uint8  
 8   HomePlanet_Europa          1739 non-null   uint8  
 9   HomePlanet_Mars            1739 non-null   uint8  
 10  CryoSleep_False            1739 non-null   uint8  
 11  CryoSleep_True             1739 non-null   uint8  
 12  Destination_55 Cancri e    1739 non-null   uint8  
 13  Destination_PSO J318.5-22  1739 non-null   u

In [12]:
#fitting model
model = LogisticRegression(max_iter=2000, random_state=0)
model.fit(X_train, y_train)
predictions = model.predict(X_valid)
y_valid_transported = np.array(y_valid.reset_index()['Transported'])

accuracy_score(y_valid_transported, predictions)

0.7734330074755607

In [13]:
#different model parameters
model2 = LogisticRegression(max_iter=2000, class_weight='balanced', random_state=0)
model2.fit(X_train, y_train)
predictions = model2.predict(X_valid)
y_valid_transported = np.array(y_valid.reset_index()['Transported'])

accuracy_score(y_valid_transported, predictions)

0.777458309373203

In [14]:
#different model
model3 = SGDClassifier(random_state=0)
model3.fit(X_train, y_train)
predictions = model3.predict(X_valid)
y_valid_transported = np.array(y_valid.reset_index()['Transported'])

accuracy_score(y_valid_transported, predictions)

0.5710178263369753

Since the first model has the highest accuracy, I'll use it to predict values of the test set

In [15]:
# predicting y_test
y_test = pd.DataFrame(model.predict(X_test))
y_test.columns = ['Transported']
y_test.index = X_test.index
y_test

,Transported
PassengerId,
0013_01,True
0018_01,False
0019_01,True
0021_01,True
0023_01,True
...,...
9266_02,True
9269_01,False
9271_01,True


In [16]:
y_test.to_csv('/kaggle/working/test_predictions.csv')